# spark-bestfit API Demo (LocalBackend)

This notebook demonstrates the complete API for the `spark-bestfit` library using the **LocalBackend** - no Spark required!

The LocalBackend uses Python's `ThreadPoolExecutor` for parallel distribution fitting, making it perfect for:
- Development and testing without Spark
- Small to medium datasets (< 1M rows)
- Quick prototyping before scaling to Spark

**Topics covered:**

1. **Setup** - LocalBackend initialization and data preparation
2. **Excluded Distributions** - Customizing which distributions to fit
3. **Distribution Fitting** - Basic and advanced fitting options
4. **FitterConfig Builder** - Fluent configuration API (v2.2.0+)
5. **Progress Tracking** - Monitor long-running fits with callbacks
6. **Working with Results** - FitResults and DistributionFitResult objects
7. **Lazy Metrics** - Skip KS/AD computation for faster model selection (v1.5.0+)
8. **Pre-filtering** - Skip incompatible distributions for faster fitting (v1.6.0+)
9. **Plotting** - Visualization with Q-Q and P-P plots
10. **Multi-Column Fitting** - Fit multiple columns in one operation
11. **Serialization** - Save and load fitted distributions

## Setup

The LocalBackend works with **pandas DataFrames** and uses Python threads for parallelism.
No Spark installation required!

In [ ]:
import numpy as np
import pandas as pd

# Import spark-bestfit components
from spark_bestfit import (
    DistributionFitter,
    LocalBackend,
    FitterConfig,
    FitterConfigBuilder,
    DEFAULT_EXCLUDED_DISTRIBUTIONS,
)

# Create LocalBackend - specify max_workers for thread parallelism
backend = LocalBackend(max_workers=4)
print(f"LocalBackend created with {backend.max_workers} workers")

## Generate Sample Data

We'll create sample data from known distributions for demonstration.

In [ ]:
np.random.seed(42)

# Normal distribution data
normal_data = np.random.normal(loc=50, scale=10, size=50_000)
df_normal = pd.DataFrame({"value": normal_data})

# Exponential distribution data (non-negative)
exp_data = np.random.exponential(scale=5, size=50_000)
df_exp = pd.DataFrame({"value": exp_data})

# Gamma distribution data
gamma_data = np.random.gamma(shape=2.0, scale=2.0, size=50_000)
df_gamma = pd.DataFrame({"value": gamma_data})

print(f"Normal data: {len(df_normal):,} rows, mean={normal_data.mean():.2f}, std={normal_data.std():.2f}")
print(f"Exponential data: {len(df_exp):,} rows, mean={exp_data.mean():.2f}")
print(f"Gamma data: {len(df_gamma):,} rows, mean={gamma_data.mean():.2f}")

---

# Part 1: Excluded Distributions

spark-bestfit excludes some slow distributions by default. You can customize this.

## 1.1 DEFAULT_EXCLUDED_DISTRIBUTIONS

Some distributions are excluded by default because they are very slow to fit.

In [ ]:
# View default excluded distributions
print(f"Default excluded distributions ({len(DEFAULT_EXCLUDED_DISTRIBUTIONS)}):")
for dist in sorted(DEFAULT_EXCLUDED_DISTRIBUTIONS):
    print(f"  - {dist}")

In [ ]:
# Include a specific distribution that's excluded by default
custom_exclusions = tuple(d for d in DEFAULT_EXCLUDED_DISTRIBUTIONS if d != "wald")

fitter_with_wald = DistributionFitter(backend=backend, excluded_distributions=custom_exclusions)
print(f"Now fitting 'wald' distribution (removed from exclusions)")

---

# Part 2: Distribution Fitting

The `DistributionFitter` class is the main entry point for fitting distributions.

## 2.1 Basic Fitting

In [ ]:
# Create fitter with LocalBackend
fitter = DistributionFitter(backend=backend)

# Fit distributions to normal data (limit to 20 for demo speed)
print("Fitting distributions to normal data...")
results_normal = fitter.fit(df_normal, column="value", max_distributions=20)

print(f"\nFitted {results_normal.count()} distributions")

## 2.2 Fitting with Custom Parameters

In [ ]:
# Fit only non-negative distributions using support_at_zero=True
fitter_nonneg = DistributionFitter(backend=backend)

print("Fitting non-negative distributions to exponential data...")
results_exp = fitter_nonneg.fit(
    df_exp,
    column="value",
    bins=100,
    support_at_zero=True,  # Only fit non-negative distributions
    enable_sampling=True,
    max_distributions=15,
)

print(f"Fitted {results_exp.count()} non-negative distributions")

## 2.3 FitterConfig Builder (v2.2.0+)

For complex configurations with many parameters, use the **fluent builder pattern**. This provides:
- **Cleaner code**: Grouped, readable configuration
- **Reusability**: Same config works across multiple fits  
- **IDE-friendly**: Better autocomplete and discoverability
- **Immutable**: Frozen dataclass prevents accidental mutation

In [ ]:
# Build a reusable configuration with the fluent builder
config = (FitterConfigBuilder()
    .with_bins(100)                           # Histogram bins
    .with_support_at_zero()                   # Non-negative distributions only
    .with_sampling(fraction=0.5)              # Sample 50% of data
    .with_max_distributions(15)               # Limit to 15 distributions
    .with_lazy_metrics()                      # Defer KS/AD computation
    .build())

print("FitterConfig created:")
print(f"  bins: {config.bins}")
print(f"  support_at_zero: {config.support_at_zero}")
print(f"  sample_fraction: {config.sample_fraction}")
print(f"  max_distributions: {config.max_distributions}")
print(f"  lazy_metrics: {config.lazy_metrics}")

# Use config with fitter
fitter_config = DistributionFitter(backend=backend)
results_config = fitter_config.fit(df_exp, column="value", config=config)

print(f"\nFitted {results_config.count()} distributions using FitterConfig")

# Config is reusable - use same config for different data
results_gamma_config = fitter_config.fit(df_gamma, column="value", config=config)
print(f"Fitted {results_gamma_config.count()} distributions (reused same config)")

In [ ]:
# FitterConfig with bounded fitting and prefilter
bounded_config = (FitterConfigBuilder()
    .with_bounds(lower=0, upper=100)          # Explicit bounds
    .with_prefilter()                          # Skip incompatible distributions
    .with_lazy_metrics()                       # Fast model selection
    .with_max_distributions(20)
    .build())

print("Bounded FitterConfig:")
print(f"  bounded: {bounded_config.bounded}")
print(f"  lower_bound: {bounded_config.lower_bound}")
print(f"  upper_bound: {bounded_config.upper_bound}")
print(f"  prefilter: {bounded_config.prefilter}")

# You can also create FitterConfig directly (without builder)
direct_config = FitterConfig(
    bins=50,
    lazy_metrics=True,
    max_distributions=10,
)
print(f"\nDirect FitterConfig: bins={direct_config.bins}, lazy={direct_config.lazy_metrics}")

## 2.4 Progress Tracking

For long-running fits, you can monitor progress with a callback. The easiest way is to use the built-in `console_progress()` utility:

```python
from spark_bestfit import console_progress

results = fitter.fit(df, column="value", progress_callback=console_progress())
```

For custom callbacks, pass any function matching `(completed: int, total: int, percent: float) -> None`.

In [ ]:
from spark_bestfit import console_progress

# Simple approach: use built-in console_progress()
print("Fitting with console_progress()...")
fitter_progress = DistributionFitter(backend=backend)
results_progress = fitter_progress.fit(
    df_normal,
    column="value",
    max_distributions=25,
    progress_callback=console_progress("Fitting"),  # Built-in utility
)
print()  # Newline after progress
print(f"Fitted {results_progress.count()} distributions")

---

# Part 3: Working with Results

The `fit()` method returns a `FitResults` object for easy result manipulation.

## 3.1 Getting Best Distributions

In [ ]:
# Get best distribution (by K-S statistic, the default)
best = results_normal.best(n=1)[0]
print(f"Best by K-S statistic: {best.distribution}")
print(f"  K-S statistic: {best.ks_statistic:.6f}")
print(f"  p-value: {best.pvalue:.4f}")
print(f"  A-D statistic: {best.ad_statistic:.6f}")
print(f"  A-D p-value: {best.ad_pvalue:.4f}" if best.ad_pvalue else f"  A-D p-value: N/A (not available for {best.distribution})")
print(f"  SSE: {best.sse:.6f}")
print(f"  AIC: {best.aic:.2f}")
print(f"  BIC: {best.bic:.2f}")
print(f"  Parameters: {[f'{p:.4f}' for p in best.parameters]}")

In [ ]:
# Get top 5 by different metrics
print("Top 5 by K-S statistic (default):")
for i, r in enumerate(results_normal.best(n=5), 1):
    print(f"  {i}. {r.distribution:20s} KS={r.ks_statistic:.6f} p={r.pvalue:.4f}")

print("\nTop 5 by A-D statistic:")
for i, r in enumerate(results_normal.best(n=5, metric="ad_statistic"), 1):
    ad_p = f"{r.ad_pvalue:.4f}" if r.ad_pvalue else "N/A"
    print(f"  {i}. {r.distribution:20s} AD={r.ad_statistic:.6f} p={ad_p}")

print("\nTop 5 by SSE:")
for i, r in enumerate(results_normal.best(n=5, metric="sse"), 1):
    print(f"  {i}. {r.distribution:20s} SSE={r.sse:.6f}")

print("\nTop 5 by AIC:")
for i, r in enumerate(results_normal.best(n=5, metric="aic"), 1):
    print(f"  {i}. {r.distribution:20s} AIC={r.aic:.2f}")

## 3.2 Filtering Results

In [ ]:
# Filter by K-S statistic threshold
good_fits = results_normal.filter(ks_threshold=0.05)
print(f"Distributions with K-S statistic < 0.05: {good_fits.count()}")

for r in good_fits.best(n=10):
    print(f"  {r.distribution:20s} KS={r.ks_statistic:.6f} p={r.pvalue:.4f}")

# Filter by p-value threshold (keep distributions with p-value > 0.05)
significant = results_normal.filter(pvalue_threshold=0.05)
print(f"\nDistributions with p-value > 0.05: {significant.count()}")

# Filter by A-D statistic threshold
good_ad = results_normal.filter(ad_threshold=2.0)
print(f"\nDistributions with A-D statistic < 2.0: {good_ad.count()}")

## 3.3 Converting to Pandas

With LocalBackend, results are already in pandas format!

In [ ]:
# Results DataFrame is already pandas with LocalBackend
df_results = results_normal.df
print("Results as pandas DataFrame:")
df_results.head(10)

## 3.4 Using Fitted Distributions

In [ ]:
# The DistributionFitResult object wraps the scipy.stats distribution
best = results_normal.best(n=1)[0]

# Generate samples from the fitted distribution
samples = best.sample(size=10000, random_state=42)
print(f"Generated {len(samples)} samples from fitted {best.distribution}")
print(f"  Sample mean: {samples.mean():.2f} (original: {normal_data.mean():.2f})")
print(f"  Sample std: {samples.std():.2f} (original: {normal_data.std():.2f})")

In [ ]:
# Evaluate PDF at specific points
x = np.array([30, 40, 50, 60, 70])
pdf_values = best.pdf(x)
cdf_values = best.cdf(x)

print("PDF and CDF values:")
for xi, pdf, cdf in zip(x, pdf_values, cdf_values):
    print(f"  x={xi}: PDF={pdf:.6f}, CDF={cdf:.4f}")

## 3.5 Parameter Confidence Intervals

Compute bootstrap confidence intervals for fitted distribution parameters. This is useful for understanding the uncertainty in your parameter estimates.

**Note**: CI width depends on sample size and distribution identifiability. Highly flexible distributions (like beta with 4 parameters) may have wider CIs due to parameter identifiability issues.

In [ ]:
# Use exponential fit for CI demo (simpler distribution = more stable CIs)
best_exp = results_exp.best(n=1)[0]

print(f"Distribution: {best_exp.distribution}")
print(f"Parameter names: {best_exp.get_param_names()}")
print(f"Fitted values: {[f'{p:.4f}' for p in best_exp.parameters]}")

# Compute 95% bootstrap confidence intervals
print("\nComputing 95% confidence intervals (this may take a few seconds)...")
ci = best_exp.confidence_intervals(
    df_exp,
    column="value",
    alpha=0.05,           # 95% CI
    n_bootstrap=500,      # Number of bootstrap samples (use 1000+ for production)
    random_seed=42,       # For reproducibility
)

print("\nParameter confidence intervals:")
for param, (lower, upper) in ci.items():
    print(f"  {param}: [{lower:.4f}, {upper:.4f}]")

## 3.6 Lazy Metrics for Fast Model Selection (v1.5.0+)

When fitting ~100 distributions, computing KS and AD statistics for all of them can be slow. With **lazy metrics**, these expensive computations are skipped during fitting and only computed on-demand when you actually need them.

**Key benefits:**
- Fast initial fitting (skip KS/AD computation)
- On-demand computation only for distributions you access
- Ideal for model selection workflows using AIC/BIC

In [ ]:
# Fit with lazy metrics - KS/AD statistics are NOT computed during fitting
print("Fitting with lazy_metrics=True (fast)...")
fitter_lazy = DistributionFitter(backend=backend)
results_lazy = fitter_lazy.fit(
    df_normal,
    column="value",
    max_distributions=20,
    lazy_metrics=True,  # Skip KS/AD computation!
)

print(f"Fitted {results_lazy.count()} distributions")
print(f"Is lazy: {results_lazy.is_lazy}")

In [ ]:
# Get best by AIC - fast! No KS/AD computation needed
best_aic = results_lazy.best(n=1, metric="aic")[0]

print(f"Best by AIC: {best_aic.distribution}")
print(f"  AIC: {best_aic.aic:.2f}")
print(f"  BIC: {best_aic.bic:.2f}")
print(f"  KS statistic: {best_aic.ks_statistic}")  # None - not computed yet!
print(f"  AD statistic: {best_aic.ad_statistic}")  # None - not computed yet!

In [ ]:
# Get best by KS statistic - triggers ON-DEMAND computation!
# Only computes KS/AD for top candidates, not all distributions
best_ks = results_lazy.best(n=1, metric="ks_statistic")[0]

print(f"Best by KS: {best_ks.distribution}")
print(f"  KS statistic: {best_ks.ks_statistic:.6f}")  # Computed on-demand!
print(f"  p-value: {best_ks.pvalue:.4f}")
print(f"  AD statistic: {best_ks.ad_statistic:.6f}")

In [ ]:
# If you need all metrics computed (e.g., before unpersisting source data),
# use materialize() to force computation of all KS/AD statistics
materialized = results_lazy.materialize()

print(f"Is lazy after materialize: {materialized.is_lazy}")  # False

# Now all distributions have KS/AD computed
top_3 = materialized.best(n=3, metric="ks_statistic")
print("\nTop 3 distributions (all metrics available):")
for i, r in enumerate(top_3, 1):
    print(f"  {i}. {r.distribution:15} KS={r.ks_statistic:.6f} p={r.pvalue:.4f}")

## 3.7 Pre-filtering Distributions (v1.6.0+)

When you know something about your data, you can skip distributions that are mathematically incompatible. Pre-filtering uses data characteristics (support bounds, skewness, kurtosis) to eliminate distributions before the expensive fitting step.

**Filtering layers:**
1. **Support bounds (100% reliable)**: Skips distributions whose support doesn't contain your data range
2. **Skewness sign (95% reliable)**: Skips positive-skew-only distributions for left-skewed data  
3. **Kurtosis (aggressive mode, ~80% reliable)**: Skips low-kurtosis distributions for heavy-tailed data

In [ ]:
# Create negative data (will filter out non-negative distributions like expon, gamma)
np.random.seed(42)
negative_data = np.random.normal(loc=-50, scale=10, size=10_000)
df_negative = pd.DataFrame({"value": negative_data})

print(f"Data range: [{negative_data.min():.1f}, {negative_data.max():.1f}]")
print(f"All values are negative - expon/gamma distributions cannot fit this data")

In [ ]:
# Fit WITHOUT prefilter (baseline)
print("Fitting WITHOUT prefilter...")
fitter_no_prefilter = DistributionFitter(backend=backend)
results_no_prefilter = fitter_no_prefilter.fit(
    df_negative, 
    column="value", 
    max_distributions=20,
    prefilter=False,  # Default - fit all distributions
)
print(f"Fitted {results_no_prefilter.count()} distributions (no prefilter)")

In [ ]:
# Fit WITH prefilter (safe mode) - skips incompatible distributions
print("\nFitting WITH prefilter=True (safe mode)...")
fitter_prefilter = DistributionFitter(backend=backend)
results_prefilter = fitter_prefilter.fit(
    df_negative,
    column="value", 
    max_distributions=20,
    prefilter=True,  # Enable pre-filtering!
)
print(f"Fitted {results_prefilter.count()} distributions (with prefilter)")
print("\n-> Pre-filter skipped distributions incompatible with negative data")

In [ ]:
# Compare best fits - both should find similar results for normal data
best_no_prefilter = results_no_prefilter.best(n=1)[0]
best_prefilter = results_prefilter.best(n=1)[0]

print("Best distribution comparison:")
print(f"  Without prefilter: {best_no_prefilter.distribution} (KS={best_no_prefilter.ks_statistic:.6f})")
print(f"  With prefilter:    {best_prefilter.distribution} (KS={best_prefilter.ks_statistic:.6f})")
print("\n-> Same best fit, but prefilter was faster by skipping incompatible distributions!")

---

# Part 4: Plotting

Visualize the fitted distribution with the data histogram.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

## 4.1 Basic Plot

In [ ]:
# Basic plot with default config
fig, ax = fitter.plot(
    best,
    df_normal,
    "value",
    title="Best Fit Distribution (Normal Data)",
    xlabel="Value",
    ylabel="Density"
)
plt.show()

## 4.2 Plot with Custom Parameters

In [ ]:
# Custom plot with direct parameters
fig, ax = fitter.plot(
    best,
    df_normal,
    "value",
    figsize=(14, 8),
    dpi=100,
    histogram_alpha=0.7,
    pdf_linewidth=3,
    title_fontsize=18,
    label_fontsize=14,
    legend_fontsize=12,
    grid_alpha=0.4,
    title="Distribution Fit with Custom Styling",
    xlabel="Value",
    ylabel="Density",
)
plt.show()

## 4.3 Plot Non-Negative Distribution

In [ ]:
# Best fit for exponential data
best_exp = results_exp.best(n=1)[0]
print(f"Best fit for exponential data: {best_exp.distribution}")
print(f"  K-S statistic: {best_exp.ks_statistic:.6f}")
print(f"  p-value: {best_exp.pvalue:.4f}")

fig, ax = fitter_nonneg.plot(
    best_exp,
    df_exp,
    "value",
    figsize=(14, 8),
    dpi=100,
    histogram_alpha=0.7,
    pdf_linewidth=3,
    title_fontsize=18,
    title=f"Best Fit: {best_exp.distribution.capitalize()}",
    xlabel="Value",
    ylabel="Density",
)
plt.show()

## 4.4 Q-Q Plots for Goodness-of-Fit Assessment

A Q-Q (quantile-quantile) plot is a powerful visual tool for assessing how well a distribution fits your data. It plots sample quantiles against theoretical quantiles from the fitted distribution. If the fit is good, points will fall approximately along the diagonal reference line.

In [ ]:
# Q-Q plot for the best fit on normal data
fig, ax = fitter.plot_qq(
    best,
    df_normal,
    "value",
    max_points=1000,  # Sample size for plotting (too many points clutters the plot)
    figsize=(10, 10),
    title="Q-Q Plot: Normal Data vs Fitted Distribution",
)
plt.show()

# Compare: Q-Q plot for exponential data
fig, ax = fitter_nonneg.plot_qq(
    best_exp,
    df_exp,
    "value",
    max_points=1000,
    figsize=(10, 10),
    title="Q-Q Plot: Exponential Data vs Fitted Distribution",
)
plt.show()

## 4.5 P-P Plots for Goodness-of-Fit Assessment

A P-P (probability-probability) plot compares the empirical cumulative distribution function (CDF) of the sample data against the theoretical CDF of the fitted distribution. It is particularly useful for assessing the fit in the center of the distribution.

In [ ]:
# P-P plot for the best fit on normal data
fig, ax = fitter.plot_pp(
    best,
    df_normal,
    "value",
    max_points=1000,  # Sample size for plotting (too many points clutters the plot)
    figsize=(10, 10),
    title="P-P Plot: Normal Data vs Fitted Distribution",
)
plt.show()

# Compare: P-P plot for exponential data
fig, ax = fitter_nonneg.plot_pp(
    best_exp,
    df_exp,
    "value",
    max_points=1000,
    figsize=(10, 10),
    title="P-P Plot: Exponential Data vs Fitted Distribution",
)
plt.show()

---

# Part 5: Multi-Column Fitting

Fit multiple columns efficiently in a single operation. This shares overhead (sampling, broadcasting) across all columns.

## 5.1 Create Multi-Column DataFrame

In [ ]:
# Create a DataFrame with multiple columns (different distributions)
np.random.seed(42)

n_rows = 20_000
df_multi = pd.DataFrame({
    "normal_col": np.random.normal(50, 10, n_rows),
    "exp_col": np.random.exponential(5, n_rows),
    "gamma_col": np.random.gamma(2.0, 2.0, n_rows),
})

print(f"Created DataFrame with {len(df_multi):,} rows and columns: {list(df_multi.columns)}")

## 5.2 Fit Multiple Columns in One Call

In [ ]:
# Fit distributions to all columns in a single operation
# This is more efficient than fitting each column separately
print("Fitting distributions to 3 columns simultaneously...")

fitter_multi = DistributionFitter(backend=backend)
results_multi = fitter_multi.fit(
    df_multi,
    columns=["normal_col", "exp_col", "gamma_col"],  # Multi-column fitting!
    max_distributions=15,
)

print(f"\nTotal results: {results_multi.count()}")
print(f"Columns fitted: {results_multi.column_names}")

## 5.3 Get Best Distribution Per Column

In [ ]:
# Get the best distribution for each column
best_per_col = results_multi.best_per_column(n=1)

print("Best distribution per column:")
for col_name, fits in best_per_col.items():
    best = fits[0]
    print(f"\n{col_name}:")
    print(f"  Distribution: {best.distribution}")
    print(f"  K-S statistic: {best.ks_statistic:.6f}")
    print(f"  p-value: {best.pvalue:.4f}")
    print(f"  Parameters: {[f'{p:.4f}' for p in best.parameters]}")

## 5.4 Filter Results by Column

In [ ]:
# Get results for a specific column
exp_results = results_multi.for_column("exp_col")

print(f"Results for 'exp_col': {exp_results.count()} distributions")
print("\nTop 5 by K-S statistic:")
for i, r in enumerate(exp_results.best(n=5), 1):
    print(f"  {i}. {r.distribution:15} KS={r.ks_statistic:.6f}")

## 5.5 Plot Results for Each Column

In [ ]:
# Plot the best fit for each column
for col_name, fits in best_per_col.items():
    best = fits[0]
    fig, ax = fitter_multi.plot(
        best,
        df_multi,
        col_name,
        title=f"{col_name}: {best.distribution} (KS={best.ks_statistic:.4f})",
        xlabel="Value",
        ylabel="Density",
        figsize=(10, 6),
    )
    plt.show()

---

# Part 6: Serialization

Save fitted distributions to disk and reload them later for inference without re-fitting.

## 6.1 Save and Load

Save a fitted distribution to JSON (default) or pickle format.

In [ ]:
import tempfile
import os
from pathlib import Path

# Use the best fit from normal data
best = results_normal.best(n=1)[0]
print(f"Saving distribution: {best.distribution}")
print(f"Parameters: {best.parameters}")

# Save to a temporary directory
model_dir = Path(tempfile.mkdtemp())
json_path = model_dir / "best_model.json"
pkl_path = model_dir / "best_model.pkl"

# Save as JSON (human-readable, default)
best.save(json_path)
print(f"\nSaved to JSON: {json_path}")
print(f"File size: {json_path.stat().st_size:,} bytes")

# Save as pickle (binary, faster)
best.save(pkl_path, format="pickle")
print(f"\nSaved to pickle: {pkl_path}")
print(f"File size: {pkl_path.stat().st_size:,} bytes")

In [ ]:
from spark_bestfit import DistributionFitResult

# Load the saved model
loaded = DistributionFitResult.load(json_path)

print(f"Loaded distribution: {loaded.distribution}")
print(f"Parameters: {loaded.parameters}")
print(f"K-S statistic: {loaded.ks_statistic:.6f}")
print(f"p-value: {loaded.pvalue:.4f}")

# Verify loaded model works
samples = loaded.sample(size=1000, random_state=42)
print(f"\nGenerated {len(samples)} samples from loaded model")
print(f"Sample mean: {samples.mean():.2f}")
print(f"Sample std: {samples.std():.2f}")

## 6.2 Data Summary

When fitting with `DistributionFitter`, the result includes data statistics
about the fitted data. This provides lightweight provenance tracking.

In [ ]:
# Access data statistics from the loaded model (v2.0+ flat field API)
if loaded.data_count is not None:
    print("Data Statistics (from fitting):")
    print(f"  Sample size: {loaded.data_count:,.0f}")
    print(f"  Min: {loaded.data_min:.4f}")
    print(f"  Max: {loaded.data_max:.4f}")
    print(f"  Mean: {loaded.data_mean:.4f}")
    print(f"  Std: {loaded.data_stddev:.4f}")
else:
    print("No data statistics available (result may have been created manually)")

## 6.3 JSON Format

The JSON format is human-readable and includes version metadata for compatibility.

In [ ]:
# View the JSON content
with open(json_path) as f:
    content = f.read()

print("JSON file content:")
print(content)

In [ ]:
# Cleanup temporary files
import shutil
shutil.rmtree(model_dir)
print(f"Cleaned up temporary directory: {model_dir}")

---

## Summary

This notebook demonstrated the complete spark-bestfit API using the **LocalBackend**:

### LocalBackend Benefits
- **No Spark required** - Works with pure Python and pandas
- **Thread-based parallelism** - Uses `ThreadPoolExecutor` for parallel fitting
- **Same API** - Identical to SparkBackend for easy migration
- **Fast for small data** - Lower overhead than Spark for < 1M rows

### Key Features Covered

1. **Setup**: `LocalBackend(max_workers=4)` for thread-based parallelism

2. **Excluded Distributions**: `DEFAULT_EXCLUDED_DISTRIBUTIONS` and custom exclusions

3. **Fitting**: `DistributionFitter(backend=backend).fit(df, column="value", ...)`

4. **FitterConfig Builder (v2.2.0+)**: Fluent API for complex configurations

5. **Progress Tracking**: `progress_callback=console_progress("Fitting")`

6. **Results**:
   - `results.best(n, metric)` - Get top N by K-S, A-D, SSE, AIC, or BIC
   - `results.filter(ks_threshold, pvalue_threshold, ad_threshold)`
   - `results.df` - Already a pandas DataFrame with LocalBackend!

7. **Using Fitted Distributions**:
   - `result.sample(size, random_state)` - Generate random samples
   - `result.pdf(x)`, `result.cdf(x)` - Evaluate density/CDF
   - `result.confidence_intervals(df, column, ...)` - Bootstrap CIs

8. **Lazy Metrics (v1.5.0+)**: `lazy_metrics=True` for fast model selection

9. **Pre-filtering (v1.6.0+)**: `prefilter=True` to skip incompatible distributions

10. **Plotting**:
    - `fitter.plot()` - Histogram + PDF overlay
    - `fitter.plot_qq()` - Q-Q plot for goodness-of-fit
    - `fitter.plot_pp()` - P-P plot for fit assessment

11. **Multi-Column Fitting**:
    - `fitter.fit(df, columns=[...])`
    - `results.best_per_column(n)`
    - `results.for_column(name)`

12. **Serialization**:
    - `result.save(path)` - JSON or pickle format
    - `DistributionFitResult.load(path)` - Reload saved models

### When to Use LocalBackend vs SparkBackend

| Use Case | Recommended Backend |
|----------|--------------------|
| Development/testing | LocalBackend |
| < 1M rows | LocalBackend |
| > 10M rows | SparkBackend |
| Production cluster | SparkBackend |
| Quick prototyping | LocalBackend |
| No Spark available | LocalBackend |